In [32]:
import numpy as np
import pandas as pd

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.datasets import make_regression

In [34]:
def metric(answers, user_csv):

    delta_c = np.abs(np.array(answers['C']) - np.array(user_csv['C']))
    hit_rate_c = np.int64(delta_c < 0.02)

    delta_t = np.abs(np.array(answers['TST']) - np.array(user_csv['TST']))
    hit_rate_t = np.int64(delta_t < 20)

    N = np.size(answers['C'])

    return np.sum(hit_rate_c + hit_rate_t) / 2 / N

In [35]:
X, y = make_regression(n_samples=10000, n_features=45, n_informative=45, random_state=1)
_ , y2 = make_regression(n_samples=10000, n_features=45, n_informative=40, random_state=42)

dummy_df = pd.concat([pd.DataFrame(X),
                      pd.DataFrame(y).rename(columns={0: 'C'}),
                      pd.DataFrame(y2).rename(columns={0: 'TST'})], axis=1)

In [36]:
sc = StandardScaler()
#sc = MinMaxScaler()

In [37]:
features_c = np.array(dummy_df.iloc[:, :45])
target_c = np.array(dummy_df.iloc[:, 45:46]) #C

features_train_c, features_test_c, target_train_c, target_test_c = train_test_split(features_c, target_c, random_state=0)

baseline_c = Ridge()

features_train_c = sc.fit_transform(features_train_c)
features_test_c = sc.fit_transform(features_test_c)

baseline_c.fit(features_train_c, target_train_c)

baseline_c.score(features_test_c, target_test_c)

0.9945595215268416

In [38]:
features_tst = np.array(dummy_df.iloc[:, :45])
target_tst = np.array(dummy_df.iloc[:, 46:47]) #TST

features_train_tst, features_test_tst, target_train_tst, target_test_tst = train_test_split(features_tst, target_tst, random_state=0)

baseline_tst = Ridge()

features_train_tst = sc.fit_transform(features_train_tst)
features_test_tst = sc.fit_transform(features_test_tst)

baseline_tst.fit(features_train_tst, target_train_tst)

baseline_tst.score(features_test_tst, target_test_tst)

-0.007406773000429245

In [39]:
model_tst = baseline_tst.fit(features_train_c, target_train_c)

In [40]:
model_c = baseline_c.fit(features_train_c, target_train_c)

In [41]:
metric(
    pd.concat([
        pd.DataFrame(model_c.predict(features_test_c)).rename(columns={0: 'C'}),
        pd.DataFrame(model_tst.predict(features_test_tst)).rename(columns={0: 'TST'})],
        axis=1),
    pd.concat([
        pd.DataFrame(target_test_c).rename(columns={0: 'C'}),
        pd.DataFrame(target_test_tst).rename(columns={0: 'TST'})],
        axis=1),
    )

0.0134